In [ ]:
import os
from diarize import srt_to_transcript
from transcriptTools import (
    loadSpeakerLabels,
    transcript_to_srt,
)
from speakerTools import (
    getReferenceLabel,
    getSpeakerLabels,
)

ROOT = os.getcwd()

reference = os.path.join(ROOT, "john.wav")
codedlabels = loadSpeakerLabels("speaker_labels.csv")

transcriptsDir = os.path.join(ROOT, "transcripts")
wavsDir = os.path.join(ROOT, "split_wavs")

transcriptFiles = [
    (
        os.path.join(transcriptsDir, showname, filename),
        showname,
        filename.split("_-_")[0],
        filename,
    )
    for showname in sorted(os.listdir(transcriptsDir))
    for filename in sorted(os.listdir(os.path.join(transcriptsDir, showname)))
]

hosts = {"rotl": ("John", "Merlin"), "roadwork": ("John", "Dan")}

In [ ]:
labeledDir = os.path.join(ROOT, "labeled")
os.makedirs(labeledDir, exist_ok=True)
os.makedirs(os.path.join(labeledDir, "rotl"), exist_ok=True)
os.makedirs(os.path.join(labeledDir, "roadwork"), exist_ok=True)

mislabeled = []

for filepath, showname, episode, filename in transcriptFiles:
    showDir = os.path.join(wavsDir, showname, episode)
    wavFiles = [os.path.join(showDir, file) for file in os.listdir(showDir)]
    referenceLabel = getReferenceLabel(reference, wavFiles)
    inferredLabels = getSpeakerLabels(referenceLabel, hosts[showname])
    isMatch = True if codedlabels[showname][episode] == inferredLabels else False
    if not isMatch:
        print(f"{showname} - {episode}")
        mislabeled.append(f"{showname} - {episode}")
    transcript = srt_to_transcript(filepath)
    labeled = [
        (idx, start, end, inferredLabels[speaker], speech)
        for idx, start, end, speaker, speech in transcript
    ]
    srt = transcript_to_srt(labeled)
    outpath = os.path.join(labeledDir, showname, filename)
    f = open(outpath, "w")
    f.write(srt)
    f.close()
f = open("mislabeled.txt", "w")
f.write("\n".join(mislabeled))
f.close()